<a href="https://colab.research.google.com/github/sanjanasrinivas22/1BM23CS301-AI/blob/main/AlphaBeta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Alpha-Beta pruning with tracing and automatic best-maximizer selection
# Copy into Colab and run
print("Sanjana Srinivas-1BM23CS301")
from typing import List, Optional, Tuple
import math

class Node:
    def __init__(self, name: str, children: Optional[List["Node"]] = None, value: Optional[int] = None):
        self.name = name
        self.children = children or []
        self.value = value

    def is_leaf(self):
        return self.value is not None

    def __repr__(self):
        if self.is_leaf():
            return f"{self.name}({self.value})"
        return f"{self.name}"

# Globals used for tracing/pruning reporting
EXPANSION_ORDER: List[str] = []
PRUNED_SUBTREES: List[Tuple[str, List[str]]] = []

def alpha_beta(node: Node,
               alpha: float,
               beta: float,
               maximizing_player: bool,
               path: Optional[List[str]] = None) -> Tuple[float, List[str]]:
    """
    Returns tuple (value, best_path_from_current_node_to_leaf).
    Also updates EXPANSION_ORDER and PRUNED_SUBTREES globals for tracing.
    """
    if path is None:
        path = []
    path = path + [node.name]

    # leaf
    if node.is_leaf():
        EXPANSION_ORDER.append(node.name)
        return node.value, path

    EXPANSION_ORDER.append(node.name)

    if maximizing_player:
        value = -math.inf
        best_path: List[str] = []
        for child in node.children:
            # pruning check (value >= beta) handled after exploring children values
            if value >= beta:
                PRUNED_SUBTREES.append((child.name, path + [child.name]))
                continue

            child_val, child_path = alpha_beta(child, alpha, beta, False, path)
            if child_val > value:
                value = child_val
                best_path = child_path
            alpha = max(alpha, value)
        return value, best_path
    else:
        value = math.inf
        best_path: List[str] = []
        for child in node.children:
            if value <= alpha:
                PRUNED_SUBTREES.append((child.name, path + [child.name]))
                continue

            child_val, child_path = alpha_beta(child, alpha, beta, True, path)
            if child_val < value:
                value = child_val
                best_path = child_path
            beta = min(beta, value)
        return value, best_path

# Example tree builder (same as before)
def build_example_tree() -> Node:
    L_D1 = Node("D1", value=3)
    L_D2 = Node("D2", value=5)
    L_E1 = Node("E1", value=6)
    L_E2 = Node("E2", value=9)

    L_F1 = Node("F1", value=1)
    L_F2 = Node("F2", value=2)
    L_G1 = Node("G1", value=0)
    L_G2 = Node("G2", value=-1)

    L_I1 = Node("I1", value=4)
    L_I2 = Node("I2", value=7)
    L_J1 = Node("J1", value=8)
    L_J2 = Node("J2", value=-2)

    D = Node("D", children=[L_D1, L_D2])   # 5
    E = Node("E", children=[L_E1, L_E2])   # 9

    F = Node("F", children=[L_F1, L_F2])   # 2
    G = Node("G", children=[L_G1, L_G2])   # 0

    I = Node("I", children=[L_I1, L_I2])   # 7
    J = Node("J", children=[L_J1, L_J2])   # 8

    B = Node("B", children=[D, E])         # min(5,9)=5
    C = Node("C", children=[F, G])         # min(2,0)=0
    H = Node("H", children=[I, J])         # min(7,8)=7

    A = Node("A", children=[B, C, H])      # MAX root
    return A

# Helper to evaluate a subtree without affecting global trace
def evaluate_child_without_polluting_trace(child: Node, root_name: str) -> float:
    """
    Temporarily clears global traces, evaluates the child,
    then restores the previous global traces so the final demo trace remains clean.
    This ensures we can find the best immediate child for root (MAX) without polluting output.
    """
    global EXPANSION_ORDER, PRUNED_SUBTREES
    saved_exp = EXPANSION_ORDER.copy()
    saved_pruned = PRUNED_SUBTREES.copy()

    # Clear globals for a clean evaluation
    EXPANSION_ORDER.clear()
    PRUNED_SUBTREES.clear()

    # Evaluate the child as a minimizer (since root is MAX and children are MIN)
    val, _ = alpha_beta(child, alpha=-math.inf, beta=math.inf, maximizing_player=False, path=[root_name])

    # Discard temporary expansions, restore saved
    EXPANSION_ORDER.clear()
    EXPANSION_ORDER.extend(saved_exp)
    PRUNED_SUBTREES.clear()
    PRUNED_SUBTREES.extend(saved_pruned)

    return val

# Runner / Demo
def run_demo():
    global EXPANSION_ORDER, PRUNED_SUBTREES
    EXPANSION_ORDER = []
    PRUNED_SUBTREES = []

    root = build_example_tree()

    # 1) Determine best immediate child (maximizer move) without polluting the trace
    child_values = []
    for child in root.children:
        val = evaluate_child_without_polluting_trace(child, root.name)
        child_values.append((child.name, val))

    # choose best (max) child
    best_child_name, best_child_val = max(child_values, key=lambda x: x[1])
    print(f"Best move for MAX: {best_child_name} (value = {best_child_val})\n")

    # 2) Run full alpha-beta from root and produce trace (expansions, prunings)
    EXPANSION_ORDER = []
    PRUNED_SUBTREES = []
    root_value, best_path = alpha_beta(root, alpha=-math.inf, beta=math.inf, maximizing_player=True)

    print("Alpha-Beta Pruning Demo (root is MAX):\n")
    print(f"Root value computed: {root_value}")
    print(f"Best path from root to leaf (names): {' -> '.join(best_path)}")

    print("\nNodes expanded in visit order:")
    print("  ", " , ".join(EXPANSION_ORDER))

    if PRUNED_SUBTREES:
        print("\nPruned subtrees (roots and their path from root):")
        for name, path in PRUNED_SUBTREES:
            print(f"  Pruned subtree root: {name}, path: {' -> '.join(path)}")
    else:
        print("\nNo pruning occurred for this ordering.")

    # Also display per-child values (useful to see why that child was chosen)
    print("\nImmediate child values at root (MIN nodes evaluated):")
    for name, val in child_values:
        print(f"  Child {name}: {val}")

if __name__ == "__main__":
    run_demo()


Sanjana Srinivas-1BM23CS301
Best move for MAX: H (value = 7)

Alpha-Beta Pruning Demo (root is MAX):

Root value computed: 7
Best path from root to leaf (names): A -> H -> I -> I2

Nodes expanded in visit order:
   A , B , D , D1 , D2 , E , E1 , C , F , F1 , F2 , H , I , I1 , I2 , J , J1

Pruned subtrees (roots and their path from root):
  Pruned subtree root: E2, path: A -> B -> E -> E2
  Pruned subtree root: G, path: A -> C -> G
  Pruned subtree root: J2, path: A -> H -> J -> J2

Immediate child values at root (MIN nodes evaluated):
  Child B: 5
  Child C: 0
  Child H: 7
